In [20]:
import math
import torch
import numpy as np
import gpytorch
import pandas as pd
from matplotlib import pyplot as plt
import random
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

In [3]:
#normalizing the inputs
xmeans = x_pd.mean(axis=0)
xstddv = x_pd.std(axis=0)
x_pd_normal = (x_pd - xmeans)/xstddv

x_pd = x_pd_normal

In [4]:
#normalize objective value data
y_obj_pd = y_pd.iloc[:, [0,1,2]]
ymeans = y_obj_pd.mean(axis=0)
ystddv = y_obj_pd.std(axis=0)
y_obj_pd_normal = (y_obj_pd - ymeans) / ystddv

y_pd.iloc[:, [0,1,2]] = y_obj_pd_normal

In [5]:
validation_idx = [1,7,15]

train_x_pd = x_pd.drop(validation_idx)
train_y_pd = y_pd.drop(validation_idx)

In [6]:
#make torch tensors
train_x = torch.tensor(train_x_pd.values, dtype=torch.float)
train_y1 = torch.tensor(train_y_pd['Polymer Solubility', 'mg/mL'].values, dtype=torch.float).squeeze()
train_y2 = torch.tensor(train_y_pd['Gelation Enthalpy', 'J/g'].values, dtype=torch.float).squeeze()
train_y3 = torch.tensor(train_y_pd['Shear Modulus', 'Kpa'].values, dtype=torch.float).squeeze()
train_y4 = torch.tensor(train_y_pd['Manufacturability', '--'].values, dtype=torch.long).squeeze()

In [7]:
test_x = torch.tensor(x_pd.values, dtype=torch.float)
test_y = torch.tensor(y_pd.values, dtype=torch.float).squeeze()

In [8]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# function to optimize parameters of the regression GP -
def train_reg_gp(model, likelihood, train_x, train_y, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        if i - 1  == training_iter:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))
        optimizer.step()

    return model, likelihood 

In [9]:
class DirichletGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# function to optimize parameters of the classification GP - 
def train_cls_gp(model, likelihood, train_x, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, likelihood.transformed_targets).sum()
        loss.backward()
        optimizer.step()

    return model, likelihood


In [10]:
#define likelihood and initialize model - regression
reg_likl = gpytorch.likelihoods.GaussianLikelihood()
reg_model1 = ExactGPModel(train_x, train_y1, reg_likl)
reg_model2 = ExactGPModel(train_x, train_y2, reg_likl)
reg_model3 = ExactGPModel(train_x, train_y3, reg_likl)

#define likelihood and initialize model - classification
cls_likl = gpytorch.likelihoods.DirichletClassificationLikelihood(train_y4, learn_additional_noise=False, alpha_epsilon=1e-4)
cls_model = DirichletGPModel(train_x, cls_likl.transformed_targets, cls_likl, num_classes=cls_likl.num_classes)

In [11]:
training_iter = 50

#train model - regression
reg_model1, reg_likl1 = train_reg_gp(reg_model1, reg_likl, train_x, train_y1, training_iter)
reg_model2, reg_likl2 = train_reg_gp(reg_model2, reg_likl, train_x, train_y2, training_iter)
reg_model3, reg_likl3 = train_reg_gp(reg_model3, reg_likl, train_x, train_y3, training_iter)

#train model - regression
cls_model, cls_likl = train_cls_gp(cls_model, cls_likl, train_x, training_iter)

In [12]:
def predict(model, likl, X):
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        pred = likl(model(X))
        mean = pred.mean
        std = pred.stddev
    return mean, std

In [13]:
#sum the available models to convert mutli obj problem to single obj
def multi_objective_predict(X, models, liklihoods):
    mean = 0
    var = 0
    for model in models:
        tmp_mean, tmp_std = predict(model, liklihoods[0], torch.tensor(X, dtype=torch.float))
        mean += tmp_mean
        var += tmp_std**2
    return mean, var**0.5

In [29]:
# function to calculate acquisition function 
def acquisition_func(X, models, liklihoods, y_min_curr):
    X = np.reshape(X, (1, -1))
    pred_mean, pred_std = multi_objective_predict(X, models, liklihoods) 
    improv = -y_min_curr + pred_mean
    z_score = np.divide(improv, pred_std + 1E-9)
    acf = np.multiply(improv, norm.cdf(z_score)) + np.multiply(pred_std, norm.pdf(z_score))
    return (-1.0) * acf 

In [34]:
# search for best point

x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
search_grid = np.random.uniform(x_bounds[:, 0], x_bounds[:, 1], size=(100, len(x_bounds)))

mo_models = [reg_model1, reg_model2]
mo_likls = [reg_likl1, reg_likl2]

reg_model1.eval()
reg_model2.eval()
reg_likl1.eval()
reg_likl2.eval()

y_best_curr = torch.max(train_y1 + train_y2).item()

acf_vals = [acquisition_func(search_grid[i, :].reshape(1, -1), mo_models, mo_likls, y_best_curr) for i in range(100)]\

acf_vals = np.array(acf_vals).reshape(100,)
top_idx = np.argsort(acf_vals)
search_grid = search_grid[top_idx[0:10]].reshape(10,-1)

best_acquisition_values = 1
best_x = None
for i, starting_point in enumerate(search_grid):
    res = minimize(fun=acquisition_func, 
                   x0=starting_point, 
                   bounds=x_bounds,
                   method='L-BFGS-B',
                   args=(mo_models, mo_likls, y_best_curr))
    if res.fun < best_acquisition_values:
        best_acquisition_values = res.fun
        best_x = res.x

In [37]:
best_x = torch.tensor(best_x.reshape(1,-1), dtype=torch.float)
multi_objective_predict(best_x, mo_models, mo_likls)

C:\Users\goura\AppData\Local\Temp\ipykernel_14908\4118473385.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tmp_mean, tmp_std = predict(model, liklihoods[0], torch.tensor(X, dtype=torch.float))


(tensor([0.0801]), tensor([1.7745]))

In [39]:
cls_model.eval()
cls_likl.eval()
predict(cls_model, cls_likl, best_x)

c:\Users\goura\anaconda3\envs\gpytorch\Lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(


(tensor([[-3.2414],
         [-2.1741]]),
 tensor([[2.2303],
         [2.3356]]))

In [40]:
best_x

tensor([[2.6837e+03, 8.3361e+01, 1.4722e+00, 1.3654e+04, 8.6976e+01, 4.1206e+01,
         6.6321e+01, 8.0358e+01]])